# Implement Neural Network for Keras Sample Datasets
The first section will follow the instruction of Jason Browniee's 2016 post on Neural Network with Keras. The original post is here: http://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

The only modification I made is the input size to match the dataset we had and to split the training set further into training and validation instead of using testing data for validation in Neural Network.

In [1]:
import keras

Using Theano backend.


In [2]:
import numpy as np
from keras.datasets import cifar10
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

from sklearn import preprocessing

seed = 78
np.random.seed(seed)

## Follow the Blog to do CNN and MLP for Digits Recognition

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')
X_train = X_train / 255
X_test = X_test / 255

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
def baseline_model():
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, input_dim=num_pixels, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
from sklearn.model_selection import train_test_split
X_train_feat, X_train_val, y_train_feat, y_train_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [9]:
# build the model
model = baseline_model()

# Fit the model
model.fit(X_train_feat, y_train_feat, validation_data=(X_train_val, y_train_val), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Digits: MLP Accuracy: %.2f%%" % (scores[1]*100))

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
5s - loss: 0.2965 - acc: 0.9157 - val_loss: 0.1437 - val_acc: 0.9597
Epoch 2/10
5s - loss: 0.1184 - acc: 0.9663 - val_loss: 0.1006 - val_acc: 0.9715
Epoch 3/10
5s - loss: 0.0783 - acc: 0.9772 - val_loss: 0.0887 - val_acc: 0.9735
Epoch 4/10
5s - loss: 0.0546 - acc: 0.9845 - val_loss: 0.0683 - val_acc: 0.9780
Epoch 5/10
5s - loss: 0.0395 - acc: 0.9885 - val_loss: 0.0699 - val_acc: 0.9793
Epoch 6/10
5s - loss: 0.0289 - acc: 0.9921 - val_loss: 0.0595 - val_acc: 0.9817
Epoch 7/10
5s - loss: 0.0215 - acc: 0.9945 - val_loss: 0.0600 - val_acc: 0.9807
Epoch 8/10
5s - loss: 0.0162 - acc: 0.9964 - val_loss: 0.0613 - val_acc: 0.9808
Epoch 9/10
5s - loss: 0.0115 - acc: 0.9976 - val_loss: 0.0597 - val_acc: 0.9828
Epoch 10/10
5s - loss: 0.0090 - acc: 0.9982 - val_loss: 0.0567 - val_acc: 0.9837
Digits: MLP Accuracy: 98.10%


In [10]:
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K

(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_pixels = X_train.shape[1] * X_train.shape[2]

In [11]:
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')
X_train = X_train / 255
X_test = X_test / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [12]:
def Convolutional_model():
	# create model
	model = Sequential()
	model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [14]:
X_train_feat, X_train_val, y_train_feat, y_train_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [15]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train_feat, y_train_feat, validation_data=(X_train_val, y_train_val), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Digits: Convolutional Neural Network Error: %.2f%%" % (scores[1]*100))

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
5s - loss: 0.2891 - acc: 0.9177 - val_loss: 0.1413 - val_acc: 0.9598
Epoch 2/10
5s - loss: 0.1167 - acc: 0.9665 - val_loss: 0.1022 - val_acc: 0.9700
Epoch 3/10
5s - loss: 0.0751 - acc: 0.9789 - val_loss: 0.0795 - val_acc: 0.9775
Epoch 4/10
5s - loss: 0.0535 - acc: 0.9844 - val_loss: 0.0712 - val_acc: 0.9787
Epoch 5/10
6s - loss: 0.0387 - acc: 0.9889 - val_loss: 0.0725 - val_acc: 0.9797
Epoch 6/10
5s - loss: 0.0294 - acc: 0.9918 - val_loss: 0.0632 - val_acc: 0.9823
Epoch 7/10
5s - loss: 0.0205 - acc: 0.9952 - val_loss: 0.0663 - val_acc: 0.9822
Epoch 8/10
5s - loss: 0.0157 - acc: 0.9964 - val_loss: 0.0592 - val_acc: 0.9840
Epoch 9/10
5s - loss: 0.0113 - acc: 0.9978 - val_loss: 0.0595 - val_acc: 0.9835
Epoch 10/10
5s - loss: 0.0084 - acc: 0.9985 - val_loss: 0.0649 - val_acc: 0.9825
Digits: Convolutional Neural Network Error: 97.99%


## Repeat for CIFAR10

In [16]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Design of a Multilayer Perceptron
The input to multilayer perceptron must be a vector of input. Since our input are 50000 x 32 x 32 x 3, we need to convert that to 50000 x 3072 vector.

The MLP is 1 input layer with 3072 input node, 1 hidden layer with 3072 input node, and 1 output layer with 1000 output node

The output will be converted for multiclass classification using one-hot encoding.

In [17]:
num_pixels = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_trainVec = X_train.reshape(X_train.shape[0],num_pixels).astype('float32')
X_testVec = X_test.reshape(X_test.shape[0],num_pixels).astype('float32')

In [18]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [19]:
scaler = preprocessing.MinMaxScaler()
scaler = scaler.fit(X_trainVec)
X_trainVec = scaler.transform(X_trainVec)
X_testVec = scaler.transform(X_testVec)

In [20]:
def baseline_model():
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, input_dim=num_pixels, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
X_train_feat, X_train_val, y_train_feat, y_train_val = train_test_split(X_trainVec, y_train, test_size=0.10, random_state=42)

In [24]:
model = baseline_model()
model.fit(X_train_feat, y_train_feat, validation_data=(X_train_val, y_train_val), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_testVec, y_test, verbose=0)
print("Multilayer Perceptron Accuracy: %.2f%%" % (scores[1]*100))

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
47s - loss: 13.0141 - acc: 0.1588 - val_loss: 13.1327 - val_acc: 0.1682
Epoch 2/10
50s - loss: 13.1494 - acc: 0.1650 - val_loss: 13.0881 - val_acc: 0.1670
Epoch 3/10
52s - loss: 13.0685 - acc: 0.1697 - val_loss: 13.0675 - val_acc: 0.1596
Epoch 4/10
184s - loss: 13.0928 - acc: 0.1669 - val_loss: 13.2414 - val_acc: 0.1484
Epoch 5/10
149s - loss: 13.0291 - acc: 0.1719 - val_loss: 12.9829 - val_acc: 0.1710
Epoch 6/10
149s - loss: 12.9736 - acc: 0.1753 - val_loss: 12.9443 - val_acc: 0.1770
Epoch 7/10
150s - loss: 12.9884 - acc: 0.1718 - val_loss: 12.9383 - val_acc: 0.1796
Epoch 8/10
154s - loss: 12.9684 - acc: 0.1752 - val_loss: 12.9485 - val_acc: 0.1750
Epoch 9/10
152s - loss: 12.9701 - acc: 0.1745 - val_loss: 12.9486 - val_acc: 0.1762
Epoch 10/10
154s - loss: 12.9639 - acc: 0.1758 - val_loss: 12.9688 - val_acc: 0.1716
Multilayer Perceptron Accuracy: 16.84%


## Design of a Simple Convolutional Neural Network


In [25]:
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('th')

# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [26]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_train.shape[1], X_test.shape[2], X_test.shape[3]).astype('float32')
X_train = X_train/255
X_test = X_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [27]:
def convNN():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='valid', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
X_train_feat, X_train_val, y_train_feat, y_train_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [29]:
model = convNN()
model.fit(X_train_feat, y_train_feat, validation_data=(X_train_val, y_train_val), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Convolutional Neural Network Accuracy: %.2f%%" % (scores[1]*100))

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
48s - loss: 1.7889 - acc: 0.3558 - val_loss: 1.5320 - val_acc: 0.4536
Epoch 2/10
49s - loss: 1.4267 - acc: 0.4931 - val_loss: 1.3599 - val_acc: 0.5158
Epoch 3/10
49s - loss: 1.3038 - acc: 0.5430 - val_loss: 1.2819 - val_acc: 0.5414
Epoch 4/10
65s - loss: 1.2339 - acc: 0.5699 - val_loss: 1.2214 - val_acc: 0.5708
Epoch 5/10
62s - loss: 1.1916 - acc: 0.5857 - val_loss: 1.1868 - val_acc: 0.5882
Epoch 6/10
62s - loss: 1.1549 - acc: 0.5987 - val_loss: 1.1573 - val_acc: 0.5934
Epoch 7/10
62s - loss: 1.1138 - acc: 0.6130 - val_loss: 1.1433 - val_acc: 0.5998
Epoch 8/10
62s - loss: 1.0835 - acc: 0.6243 - val_loss: 1.1167 - val_acc: 0.6132
Epoch 9/10
63s - loss: 1.0575 - acc: 0.6316 - val_loss: 1.1047 - val_acc: 0.6174
Epoch 10/10
63s - loss: 1.0356 - acc: 0.6388 - val_loss: 1.1215 - val_acc: 0.6024
Convolutional Neural Network Accuracy: 60.09%
